In [ ]:
# get env openai key 
import os
openai_key = os.getenv('OPENAI_API_KEY')
# print(openai_key)

原始模型

In [11]:
from openai import OpenAI


file = ''
with open(r'First_Phase\data\10.txt', 'r') as f:
    file = f.read()

input = file


client = OpenAI()
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "Find PHI and normalize time information"},
    {"role": "user", "content": input}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='PHI (Protected Health Information) in this text:\n\n- Lab No: 09F01654\n- D.O.B: 24/8/1993\n- Sex: M\n- Collected: 28/08/2013 at 08:26\n- Location: St Vincent-BATLOW/ADELONG MULTI PURPOSE SERVICE\n- Doctor\'s Name: Dr JAXON AL-KARSTEN\n- The patient\'s name, "SIZAR, HOWARD"\n- Specimen: Prostate and seminal vesicles\n- Histological type: Acinar adenocarcinoma\n- Tumour location: Left mid posterolateral\n- Gleason score: 3+4=7\n- Extraprostatic extension: Absent\n- Surgical margins: Clear\n- Perineural invasion: Absent\n- Seminal vesicles: Not involved\n- Lymph nodes: N/A\n- Lymphovascular invasion: Absent\n- Other: Patchy high grade prostatic intraepithelial neoplasia (PIN) is present.\n- Stage (AJCC/UICC TNM system, 7th edition): pT2c, NX, MX\n\n\nNormalized Time Information:\n- Collected: 28/08/2013 at 08:26\n\nPlease note that the time information provided is already in a normalized format.', role='assistant', function_call=None, tool_calls=None)


In [32]:
str  = completion.choices[0].message.content
# replace all '\\n' with '\n'
# str = str.replace('\\n','\n')
print(str)

PHI (Protected Health Information) in this text:

- Lab No: 09F01654
- D.O.B: 24/8/1993
- Sex: M
- Collected: 28/08/2013 at 08:26
- Location: St Vincent-BATLOW/ADELONG MULTI PURPOSE SERVICE
- Doctor's Name: Dr JAXON AL-KARSTEN
- The patient's name, "SIZAR, HOWARD"
- Specimen: Prostate and seminal vesicles
- Histological type: Acinar adenocarcinoma
- Tumour location: Left mid posterolateral
- Gleason score: 3+4=7
- Extraprostatic extension: Absent
- Surgical margins: Clear
- Perineural invasion: Absent
- Seminal vesicles: Not involved
- Lymph nodes: N/A
- Lymphovascular invasion: Absent
- Other: Patchy high grade prostatic intraepithelial neoplasia (PIN) is present.
- Stage (AJCC/UICC TNM system, 7th edition): pT2c, NX, MX


Normalized Time Information:
- Collected: 28/08/2013 at 08:26

Please note that the time information provided is already in a normalized format.


微調過模型

In [2]:
from openai import OpenAI


file = ''
with open(r'../First_Phase\data\57.txt', 'r') as f:
    file = f.read()

input = file


client = OpenAI()
completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0613:personal::8N0agg9Z",
  messages=[
    {"role": "system", "content": "Find PHI and normalize time information"},
    {"role": "user", "content": input}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='MEDICALRECORD: 009423.CMG\nPATIENT: HANE, MOHAMED\nIDNUM: 00Y42304\nDATE: 22/2/2062=>2062-02-22\nDOCTOR: JX\nDOCTOR: ZF\nDATE: 19.2.62=>2062-02-19\nDOCTOR: EQ\nDOCTOR: IH\nDATE: 20.2.62=>2062-02-20\nDOCTOR: TERENCE FREUDENTHAL\nDOCTOR: RUDOLF KATZENBACH\nDOCTOR: C Sandavol\n', role='assistant', function_call=None, tool_calls=None)


In [4]:
str  = completion.choices[0].message.content
# replace all '\\n' with '\n'
# str = str.replace('\\n','\n')
print(str)

MEDICALRECORD: 009423.CMG
PATIENT: HANE, MOHAMED
IDNUM: 00Y42304
DATE: 22/2/2062=>2062-02-22
DOCTOR: JX
DOCTOR: ZF
DATE: 19.2.62=>2062-02-19
DOCTOR: EQ
DOCTOR: IH
DATE: 20.2.62=>2062-02-20
DOCTOR: TERENCE FREUDENTHAL
DOCTOR: RUDOLF KATZENBACH
DOCTOR: C Sandavol



In [ ]:
str = '''MEDICALRECORD: 009423.CMG
PATIENT: HANE, MOHAMED
IDNUM: 00Y42304
DATE: 22/2/2062=>2062-02-22
DOCTOR: JX
DOCTOR: ZF
DATE: 19.2.62=>2062-02-19
DOCTOR: EQ
DOCTOR: IH
DATE: 20.2.62=>2062-02-20
DOCTOR: C Sandavol
DOCTOR: TERENCE FREUDENTHAL
DOCTOR: RUDOLF KATZENBACH'''

In [50]:
from Answer_file_generator import Answer_file_generator
import re
file_gen = Answer_file_generator('answer.txt')




In [51]:
gpt_res = str
now_index = 0
file_name = '57'
for line in gpt_res.split('\n'):
    #IDNUM: 00Y42304
    cilon_index = line.find(':')
    if cilon_index != -1:
        label = line[:cilon_index]
        content = line[cilon_index+2:]
        time = None
        if content.find('=>')!=-1:
            time = content[content.find('=>')+2:]
            content = content[:content.find('=>')]
            
        
        pattern = re.compile(content)
        match = pattern.search(input , pos=now_index)
        if match!=None:
            start_index = match.start()
            end_index = match.end()
            now_index = end_index
            file_gen.add_item(file_name , start_index , end_index , label , content , time)    
            
file_gen.save()